In [2]:
import xmltodict
import re
import gzip as gzip
import os
from biothings.utils.dataload import value_convert_to_number




In [4]:
current_xml = load_data('Compound_036500001_037000000.xml.gz')

In [3]:
_f = gzip.open('Compound_036500001_037000000.xml.gz','rb').read()
# doc = xmltodict.parse('Compound_036500001_037000000.xml',item_depth=2)

In [4]:
from sys import getsizeof

In [5]:
getsizeof(_f)

1389889197

In [6]:
def handle(path,item):
    item = restructure_dict(item)
    compound_list.append(item)
    return True

In [7]:
current_xml = xmltodict.parse('Compound_036500001_037000000.xml.gz',item_depth=2,item_callback=handle,xml_attribs=True)

ExpatError: syntax error: line 1, column 0

In [74]:
import xml.etree.ElementTree as ET

In [76]:
PC_count = False
inchi = False
inchikey = False  
hydrogen_bond_acceptor = False
hydrogen_bond_donor = False
rotatable_bond = False
iupac = False
logp = False
mass = False
molecular_formula = False
molecular_weight = False
smiles = False
topological = False
monoisotopic_weight = False
complexity = False


for event, elem in ET.iterparse("Compound_036500001_037000000.xml", events=("start","end")):
    prefix, has_namespace, postfix = elem.tag.partition('}')
    if has_namespace:
        elem.tag = postfix  # strip all namespaces
    if((elem.tag == "PC-CompoundType_id_cid") & (event == 'start')):     
        current_compound = {}
        compound_data = {}
        current_compound["_id"] = elem.text
        compound_data["cid"] = elem.text
        compound_data["iupac"] = {}
        compound_data["smiles"] = {}
    elif((elem.tag == "PC-Compound") & (event == 'end')):
        current_compound["pubchem"] = compound_data
        print(current_compound)
        elem.clear()
    elif((elem.tag == "PC-Compound_charge") & (event == 'start')):
        compound_data["formal_charge"] = elem.text
    elif((elem.tag == "PC-Count") & (event == 'start')):
        PC_count = True
    elif((elem.tag == "PC-Count") & (event == 'end')):
        PC_count = False
    elif(PC_count):
        if((elem.tag == "PC-Count_heavy-atom") & (event == 'start')):
            compound_data["heavy_atom_count"] = elem.text
        elif((elem.tag == "PC-Count_atom-chiral-def") & (event == 'start')):
            compound_data["defined_chiral_atom_count"] = elem.text
        elif((elem.tag == "PC-Count_bond-chiral-def") & (event == 'start')):
            compound_data["defined_chiral_bond_count"] = elem.text
        elif((elem.tag == "PC-Count_atom-chiral-undef") & (event == 'start')):
            compound_data["undefined_chiral_atom_count"] = elem.text
        elif((elem.tag == "PC-Count_bond-chiral-undef") & (event == 'start')):
            compound_data["undefined_chiral_bond_count"] = elem.text
        elif((elem.tag == "PC-Count_isotope-atom") & (event == 'start')):
            compound_data["isotope_atom_count"] = elem.text
        elif((elem.tag == "PC-Count_covalent-unit") & (event == 'start')):
            compound_data["covalent_unit_count"] = elem.text
        elif((elem.tag == "PC-Count_tautomers") & (event == 'start')):
            compound_data["tautomers_count"] = elem.text
    elif((elem.tag == "PC-Count") & (event == 'start')):
        PC_count = True
    elif((elem.tag == "PC-Count") & (event == 'end')):
        PC_count = False
        
    elif((elem.tag == "PC-Urn_label") & (event == 'start')):
        if(elem.text == 'InChI'):
            inchi = True
        elif(elem.text == 'InChIKey'):
            inchikey = True
        elif(elem.text == 'IUPAC Name'):
            iupac = True
        elif(elem.text == "Log P"):
            logp = True
        elif(elem.text == "Mass"):
            mass = True
        elif(elem.text == "Molecular Formula"):
            molecular_formula = True
        elif(elem.text == "Molecular Weight"):
            molecular_weight = True
        elif(elem.text == "SMILES"):
            smiles = True
        elif(elem.text == "Topological"):
            topological = True
        elif(elem.text == "Weight"):
            monoisotopic_weight = True
        elif(elem.text == "Compound Complexity"):
            complexity = True

    elif((elem.tag == "PC-Urn_name") & (event == 'start')):
        if(elem.text == 'Hydrogen Bond Acceptor'):
            hydrogen_bond_acceptor = True
        elif(elem.text == 'Hydrogen Bond Donor'):
            hydrogen_bond_donor = True
        elif(elem.text == 'Rotatable Bond'):
            rotatable_bond = True
        elif(iupac):
            iupac_key = elem.text  
        elif(smiles):
            smiles_key = elem.text  

    elif((elem.tag == "PC-InfoData_value_sval") & (event == 'start')):
        if(inchi):
            compound_data["inchi"] = elem.text
            inchi = False  
        elif(inchikey):
            compound_data["inchikey"] = elem.text
            inchikey = False  
        elif(iupac):
            compound_data['iupac'][iupac_key] = elem.text
            iupac = False
        elif(molecular_formula):
            compound_data["molecular_formula"] = elem.text
            molecular_formula = False
        elif(smiles):
            compound_data['smiles'][smiles_key] = elem.text
            smiles = False
    
    elif((elem.tag == "PC-InfoData_value_ival") & (event == 'start')):
        if(hydrogen_bond_acceptor):
            compound_data["hydrogen_bond_acceptor_count"] = elem.text
            hydrogen_bond_acceptor = False
        elif(hydrogen_bond_donor):
            compound_data["hydrogen_bond_donor_count"] = elem.text
            hydrogen_bond_donor = False
        elif(rotatable_bond):
            compound_data["rotatable_bond_count"] = elem.text
            rotatable_bond = False
        
    elif((elem.tag == "PC-InfoData_value_fval") & (event == 'start')):
        if(logp):
            compound_data["xlogp"] = elem.text
            logp = False
        elif(mass):
            compound_data["exact_mass"] = elem.text
            mass = False
        elif(molecular_weight): 
            compound_data["molecular_weight"] = elem.text
            molecular_weight = False
        elif(topological):
            compound_data["topological_polar_surface_area"] = elem.text
            topological = False
        elif(monoisotopic_weight):
            compound_data["monoisotopic_weight"] = elem.text
            monoisotopic_weight = False
        elif(complexity):
            compound_data["complexity"] = elem.text
            complexity = False
            
            
#     elem.clear()


{'_id': '36500004', 'pubchem': {'cid': '36500004', 'iupac': {'Allowed': '1-butyl-N-[4-(5-methyl-2-thienyl)thiazol-2-yl]-6-oxo-pyridazine-3-carboxamide', 'CAS-like Style': '1-butyl-N-[4-(5-methyl-2-thiophenyl)-2-thiazolyl]-6-oxo-3-pyridazinecarboxamide', 'Markup': '1-butyl-<I>N</I>-[4-(5-methylthiophen-2-yl)-1,3-thiazol-2-yl]-6-oxopyridazine-3-carboxamide', 'Preferred': '1-butyl-N-[4-(5-methylthiophen-2-yl)-1,3-thiazol-2-yl]-6-oxopyridazine-3-carboxamide', 'Systematic': '1-butyl-N-[4-(5-methylthiophen-2-yl)-1,3-thiazol-2-yl]-6-oxidanylidene-pyridazine-3-carboxamide', 'Traditional': '1-butyl-6-keto-N-[4-(5-methyl-2-thienyl)thiazol-2-yl]pyridazine-3-carboxamide'}, 'smiles': {'Canonical': 'CCCCN1C(=O)C=CC(=N1)C(=O)NC2=NC(=CS2)C3=CC=C(S3)C', 'Isomeric': 'CCCCN1C(=O)C=CC(=N1)C(=O)NC2=NC(=CS2)C3=CC=C(S3)C'}, 'formal_charge': '0', 'complexity': '581', 'hydrogen_bond_acceptor_count': '6', 'hydrogen_bond_donor_count': '1', 'rotatable_bond_count': '6', 'inchi': 'InChI=1S/C17H18N4O2S2/c1-3-4-9-21-

{'_id': '36501661', 'pubchem': {'cid': '36501661', 'iupac': {'Allowed': '2-[[5-[(2-chlorophenoxy)methyl]-4-phenyl-1,2,4-triazol-3-yl]sulfanyl]-1-(4-methoxyphenyl)ethanone', 'CAS-like Style': '2-[[5-[(2-chlorophenoxy)methyl]-4-phenyl-1,2,4-triazol-3-yl]thio]-1-(4-methoxyphenyl)ethanone', 'Markup': '2-[[5-[(2-chlorophenoxy)methyl]-4-phenyl-1,2,4-triazol-3-yl]sulfanyl]-1-(4-methoxyphenyl)ethanone', 'Preferred': '2-[[5-[(2-chlorophenoxy)methyl]-4-phenyl-1,2,4-triazol-3-yl]sulfanyl]-1-(4-methoxyphenyl)ethanone', 'Systematic': '2-[[5-[(2-chloranylphenoxy)methyl]-4-phenyl-1,2,4-triazol-3-yl]sulfanyl]-1-(4-methoxyphenyl)ethanone', 'Traditional': '2-[[5-[(2-chlorophenoxy)methyl]-4-phenyl-1,2,4-triazol-3-yl]thio]-1-(4-methoxyphenyl)ethanone'}, 'smiles': {'Canonical': 'COC1=CC=C(C=C1)C(=O)CSC2=NN=C(N2C3=CC=CC=C3)COC4=CC=CC=C4Cl', 'Isomeric': 'COC1=CC=C(C=C1)C(=O)CSC2=NN=C(N2C3=CC=CC=C3)COC4=CC=CC=C4Cl'}, 'formal_charge': '0', 'complexity': '587', 'hydrogen_bond_acceptor_count': '6', 'hydrogen_bon

{'_id': '36503993', 'pubchem': {'cid': '36503993', 'iupac': {'Allowed': '1-(4-bromophenyl)-N-[4-(difluoromethoxy)phenyl]cyclopropanecarboxamide', 'CAS-like Style': '1-(4-bromophenyl)-N-[4-(difluoromethoxy)phenyl]-1-cyclopropanecarboxamide', 'Markup': None, 'Preferred': '1-(4-bromophenyl)-N-[4-(difluoromethoxy)phenyl]cyclopropane-1-carboxamide', 'Systematic': 'N-[4-[bis(fluoranyl)methoxy]phenyl]-1-(4-bromophenyl)cyclopropane-1-carboxamide', 'Traditional': '1-(4-bromophenyl)-N-[4-(difluoromethoxy)phenyl]cyclopropanecarboxamide'}, 'smiles': {'Canonical': 'C1CC1(C2=CC=C(C=C2)Br)C(=O)NC3=CC=C(C=C3)OC(F)F', 'Isomeric': 'C1CC1(C2=CC=C(C=C2)Br)C(=O)NC3=CC=C(C=C3)OC(F)F'}, 'formal_charge': '0', 'complexity': '415', 'hydrogen_bond_acceptor_count': '4', 'hydrogen_bond_donor_count': '1', 'rotatable_bond_count': '5', 'inchi': 'InChI=1S/C17H14BrF2NO2/c18-12-3-1-11(2-4-12)17(9-10-17)15(22)21-13-5-7-14(8-6-13)23-16(19)20/h1-8,16H,9-10H2,(H,21,22)', 'inchikey': 'KNZUSROGMFAQMW-UHFFFAOYSA-N', 'xlogp': '

{'_id': '36505234', 'pubchem': {'cid': '36505234', 'iupac': {'Allowed': '(4-fluorophenyl)-[4-(3,4,5-triethoxybenzoyl)piperazin-1-yl]methanone', 'CAS-like Style': '(4-fluorophenyl)-[4-[oxo-(3,4,5-triethoxyphenyl)methyl]-1-piperazinyl]methanone', 'Markup': '(4-fluorophenyl)-[4-(3,4,5-triethoxybenzoyl)piperazin-1-yl]methanone', 'Preferred': '(4-fluorophenyl)-[4-(3,4,5-triethoxybenzoyl)piperazin-1-yl]methanone', 'Systematic': '(4-fluorophenyl)-[4-(3,4,5-triethoxyphenyl)carbonylpiperazin-1-yl]methanone', 'Traditional': '(4-fluorophenyl)-[4-(3,4,5-triethoxybenzoyl)piperazino]methanone'}, 'smiles': {'Canonical': 'CCOC1=CC(=CC(=C1OCC)OCC)C(=O)N2CCN(CC2)C(=O)C3=CC=C(C=C3)F', 'Isomeric': 'CCOC1=CC(=CC(=C1OCC)OCC)C(=O)N2CCN(CC2)C(=O)C3=CC=C(C=C3)F'}, 'formal_charge': '0', 'complexity': '589', 'hydrogen_bond_acceptor_count': '6', 'hydrogen_bond_donor_count': '0', 'rotatable_bond_count': '8', 'inchi': 'InChI=1S/C24H29FN2O5/c1-4-30-20-15-18(16-21(31-5-2)22(20)32-6-3)24(29)27-13-11-26(12-14-27)23(28)

{'_id': '36505579', 'pubchem': {'cid': '36505579', 'iupac': {'Allowed': '2-(3,5-dimethylphenoxy)-1-[4-(4-fluorobenzoyl)piperazin-1-yl]ethanone', 'CAS-like Style': '2-(3,5-dimethylphenoxy)-1-[4-[(4-fluorophenyl)-oxomethyl]-1-piperazinyl]ethanone', 'Markup': '2-(3,5-dimethylphenoxy)-1-[4-(4-fluorobenzoyl)piperazin-1-yl]ethanone', 'Preferred': '2-(3,5-dimethylphenoxy)-1-[4-(4-fluorobenzoyl)piperazin-1-yl]ethanone', 'Systematic': '2-(3,5-dimethylphenoxy)-1-[4-(4-fluorophenyl)carbonylpiperazin-1-yl]ethanone'}, 'smiles': {'Canonical': 'CC1=CC(=CC(=C1)OCC(=O)N2CCN(CC2)C(=O)C3=CC=C(C=C3)F)C', 'Isomeric': 'CC1=CC(=CC(=C1)OCC(=O)N2CCN(CC2)C(=O)C3=CC=C(C=C3)F)C'}, 'formal_charge': '0', 'complexity': '504', 'hydrogen_bond_acceptor_count': '4', 'hydrogen_bond_donor_count': '0', 'rotatable_bond_count': '4', 'inchi': 'InChI=1S/C21H23FN2O3/c1-15-11-16(2)13-19(12-15)27-14-20(25)23-7-9-24(10-8-23)21(26)17-3-5-18(22)6-4-17/h3-6,11-13H,7-10,14H2,1-2H3', 'inchikey': 'QRIPQENQTXEJSX-UHFFFAOYSA-N', 'xlogp': 

...

{'_id': '36515453', 'pubchem': {'cid': '36515453', 'iupac': {'Allowed': '[(3R)-2,3-dihydro-1,4-benzodioxin-3-yl]-[4-[(3-methyl-8-quinolyl)sulfonyl]piperazin-1-yl]methanone', 'CAS-like Style': '[(3R)-2,3-dihydro-1,4-benzodioxin-3-yl]-[4-[(3-methyl-8-quinolinyl)sulfonyl]-1-piperazinyl]methanone', 'Markup': '[(3<I>R</I>)-2,3-dihydro-1,4-benzodioxin-3-yl]-[4-(3-methylquinolin-8-yl)sulfonylpiperazin-1-yl]methanone', 'Preferred': '[(3R)-2,3-dihydro-1,4-benzodioxin-3-yl]-[4-(3-methylquinolin-8-yl)sulfonylpiperazin-1-yl]methanone', 'Systematic': '[(3R)-2,3-dihydro-1,4-benzodioxin-3-yl]-[4-(3-methylquinolin-8-yl)sulfonylpiperazin-1-yl]methanone', 'Traditional': '[(3R)-2,3-dihydro-1,4-benzodioxin-3-yl]-[4-[(3-methyl-8-quinolyl)sulfonyl]piperazino]methanone'}, 'smiles': {'Canonical': 'CC1=CC2=C(C(=CC=C2)S(=O)(=O)N3CCN(CC3)C(=O)C4COC5=CC=CC=C5O4)N=C1', 'Isomeric': 'CC1=CC2=C(C(=CC=C2)S(=O)(=O)N3CCN(CC3)C(=O)[C@H]4COC5=CC=CC=C5O4)N=C1'}, 'formal_charge': '0', 'complexity': '779', 'hydrogen_bond_acc

{'_id': '36515933', 'pubchem': {'cid': '36515933', 'iupac': {'Allowed': '1-[(1S)-1-(2,4-dichlorophenyl)ethyl]-3-[(6-imidazol-1-yl-3-pyridyl)methyl]urea', 'CAS-like Style': '1-[(1S)-1-(2,4-dichlorophenyl)ethyl]-3-[[6-(1-imidazolyl)-3-pyridinyl]methyl]urea', 'Markup': '1-[(1<I>S</I>)-1-(2,4-dichlorophenyl)ethyl]-3-[(6-imidazol-1-ylpyridin-3-yl)methyl]urea', 'Preferred': '1-[(1S)-1-(2,4-dichlorophenyl)ethyl]-3-[(6-imidazol-1-ylpyridin-3-yl)methyl]urea', 'Systematic': '1-[(1S)-1-(2,4-dichlorophenyl)ethyl]-3-[(6-imidazol-1-ylpyridin-3-yl)methyl]urea', 'Traditional': '1-[(1S)-1-(2,4-dichlorophenyl)ethyl]-3-[(6-imidazol-1-yl-3-pyridyl)methyl]urea'}, 'smiles': {'Canonical': 'CC(C1=C(C=C(C=C1)Cl)Cl)NC(=O)NCC2=CN=C(C=C2)N3C=CN=C3', 'Isomeric': 'C[C@@H](C1=C(C=C(C=C1)Cl)Cl)NC(=O)NCC2=CN=C(C=C2)N3C=CN=C3'}, 'formal_charge': '0', 'complexity': '469', 'hydrogen_bond_acceptor_count': '3', 'hydrogen_bond_donor_count': '2', 'rotatable_bond_count': '5', 'inchi': 'InChI=1S/C18H17Cl2N5O/c1-12(15-4-3-14(19

{'_id': '36516609', 'pubchem': {'cid': '36516609', 'iupac': {'Allowed': 'N-[(1S)-1-[4-(2-cyclopentylacetyl)piperazine-1-carbonyl]-2-methyl-propyl]-2,6-difluoro-benzamide', 'CAS-like Style': 'N-[(2S)-1-[4-(2-cyclopentyl-1-oxoethyl)-1-piperazinyl]-3-methyl-1-oxobutan-2-yl]-2,6-difluorobenzamide', 'Markup': None, 'Preferred': 'N-[(2S)-1-[4-(2-cyclopentylacetyl)piperazin-1-yl]-3-methyl-1-oxobutan-2-yl]-2,6-difluorobenzamide', 'Systematic': 'N-[(2S)-1-[4-(2-cyclopentylethanoyl)piperazin-1-yl]-3-methyl-1-oxidanylidene-butan-2-yl]-2,6-bis(fluoranyl)benzamide', 'Traditional': 'N-[(1S)-1-[4-(2-cyclopentylacetyl)piperazine-1-carbonyl]-2-methyl-propyl]-2,6-difluoro-benzamide'}, 'smiles': {'Canonical': 'CC(C)C(C(=O)N1CCN(CC1)C(=O)CC2CCCC2)NC(=O)C3=C(C=CC=C3F)F', 'Isomeric': 'CC(C)[C@@H](C(=O)N1CCN(CC1)C(=O)CC2CCCC2)NC(=O)C3=C(C=CC=C3F)F'}, 'formal_charge': '0', 'complexity': '636', 'hydrogen_bond_acceptor_count': '5', 'hydrogen_bond_donor_count': '1', 'rotatable_bond_count': '6', 'inchi': 'InChI=1

{'_id': '36517070', 'pubchem': {'cid': '36517070', 'iupac': {'Allowed': '1-allyl-3-[4-[[3,5-bis(trifluoromethyl)phenyl]sulfonylamino]phenyl]urea', 'CAS-like Style': '1-[4-[[3,5-bis(trifluoromethyl)phenyl]sulfonylamino]phenyl]-3-prop-2-enylurea', 'Markup': '1-[4-[[3,5-bis(trifluoromethyl)phenyl]sulfonylamino]phenyl]-3-prop-2-enylurea', 'Preferred': '1-[4-[[3,5-bis(trifluoromethyl)phenyl]sulfonylamino]phenyl]-3-prop-2-enylurea', 'Systematic': '1-[4-[[3,5-bis(trifluoromethyl)phenyl]sulfonylamino]phenyl]-3-prop-2-enyl-urea', 'Traditional': '1-allyl-3-[4-[[3,5-bis(trifluoromethyl)phenyl]sulfonylamino]phenyl]urea'}, 'smiles': {'Canonical': 'C=CCNC(=O)NC1=CC=C(C=C1)NS(=O)(=O)C2=CC(=CC(=C2)C(F)(F)F)C(F)(F)F', 'Isomeric': 'C=CCNC(=O)NC1=CC=C(C=C1)NS(=O)(=O)C2=CC(=CC(=C2)C(F)(F)F)C(F)(F)F'}, 'formal_charge': '0', 'complexity': '705', 'hydrogen_bond_acceptor_count': '10', 'hydrogen_bond_donor_count': '3', 'rotatable_bond_count': '6', 'inchi': 'InChI=1S/C18H15F6N3O3S/c1-2-7-25-16(28)26-13-3-5-14(6

KeyboardInterrupt: 